In [9]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
import os

import truststore
truststore.inject_into_ssl()
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import seaborn as sns
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
os.environ["APP_CLIENT_ID"] = "zizhang-chen-research-app"
os.environ["APP_CLIENT_SECRET"] = "cQPcueFP7tDrimbf8NW2GAHcHeQa"
from llm_idam_token_generator.idam_token_generator import get_idam_token
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI
import tiktoken
# OpenAI Endpoint details
OPENAI_ENDPOINT = "https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net"
OPENAI_DEPLOYMENT_MODEL = "gpt-4-32k-beta"
# OPENAI_DEPLOYMENT_MODEL = "gpt-35-turbo-16k"
OPENAI_AZURE_API_VERSION = "2023-12-01-preview"
# OPENAI_AZURE_API_VERSION = '2024-02-01'
OPENAI_TYPE="azure"

In [10]:
# help(AzureChatOpenAI)

llm = AzureChatOpenAI(
    api_key="xxx",  # This is not playing any role, but required as per OpenAI sdk. So any random could be passed.
    azure_endpoint=OPENAI_ENDPOINT,
    deployment_name=OPENAI_DEPLOYMENT_MODEL,
    openai_api_version=OPENAI_AZURE_API_VERSION,
    n = 2,
    default_headers={
        'Authorization': f'Bearer {get_idam_token()}',
        'Content-Type': 'application/json'
    }
)

PID:97126 INFO llm_idam_token_generator.idam_token_generator - Client ID: zizhang-chen-research-app - Expiry_time: 1727460923
PID:97126 INFO llm_idam_token_generator.idam_token_generator - Client ID: zizhang-chen-research-app - Generating new token.
PID:97126 INFO llm_idam_token_generator.idam_token_generator - Client ID: zizhang-chen-research-app - All required environment variables are present.
/opt/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
PID:97126 INFO llm_idam_token_generator.idam_token_generator - Client ID: zizhang-chen-research-app - IDAM Access Token is generated
/opt/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'

In [4]:
def generate_prompt_task(path):
    with open(path) as f:
        prompt = f.readlines()
    return prompt

def generate_prompt_task_cross_task(path):
    with open(path) as f:
        f = f.read()
    # prompt = '\n'.join(f)
    return f

def generate_cot(tasks):
    if tasks == 'icu':
        task_icl_name = "**EHR Task Results Influencing whether the patient will be transferred to the ICU on the same admission date**"
        task_target_name = 'predict whether the patient will be transferred to ICU on admission. Note, '
    elif tasks == 'los':
        task_icl_name = "**EHR Task Results Influencing whether the patient will be discharged within 7 days**"
        task_target_name ='predict whether the patient will be discharged in 7 days.'
    elif tasks == 'readmin':
        task_icl_name = "**EHR Task Results Influencing whether the patient will be readmitted in 30 days**"
        task_target_name = 'predict whether the patient will be readmitted in 30 days'

    cot = ["1. **Review Patient Profile:** Analyze age, sex, and medical history for any factors that significantly increase ICU transfer risk.",
           f"2. **Analyze Results from Other EHR Tasks:** Consider additional clinical result provided in section: {task_icl_name}"
           "3. **Evaluate Current Symptoms:** Consider how severe the symptoms are and whether they suggest worsening of condition or stability.",
           "4. **Assess Vital Signs:** Identify any vital signs that are outside normal ranges and what they indicate about the patient's current health status.",
           "5. **Interpret Laboratory Results:** Look for abnormalities in lab results that could signal a need for intensive care, such as organ dysfunction or severe infection.",
           "6. **Consider Medical Events Since Admission:** Determine if the medical events reported since admission suggest a trend towards improvement or deterioration.",
           f"7. **Make Prediction:** Based on the analysis, {task_target_name}", 
           "8. **Please be as conservative as possible, a false positive may cause serious problems.**"]
    
    return '\n'.join(cot)

def combin_prompt_with_cot(general_prompt, cot, medical_event):
    template_by_task = '**Task:**\n' + general_prompt + '\n'
    
    template_cot = '**Chain of Thought Analysis:**\n'+ cot + '\n'

    template_events = '**Medical Events**\n' + medical_event + '\n'

    cot_template = template_by_task + '\n' + template_cot + '\n' + '{events}'
    return cot_template, template_events

def combin_prompt_no_cot(general_prompt, medical_event):
    template_by_task = '**Task:**\n' + general_prompt + '\n'
    
    template_events = '**Medical Events**\n' + medical_event + '\n'

    cot_template = template_by_task + '\n' + '{events}'
    return cot_template, template_events

def sample_indices(seed, array_A, sample_size, min_true_count):
    if array_A.size < sample_size:
        raise ValueError("The size of array A must be at least as large as the sample size.")
    if np.count_nonzero(array_A) < min_true_count:
        raise ValueError("Array A does not contain enough True values.")
    
    np.random.seed(seed)
    
    # Indices of True and False in array_A
    true_indices = np.where(array_A)[0]
    false_indices = np.where(~array_A)[0]
    
    if len(true_indices) < min_true_count:
        raise ValueError("Array A contains fewer than the minimum required True values.")

    selected_true_indices = np.random.choice(true_indices, min_true_count, replace=False)
    
    # Remaining indices needed to be picked
    remaining_indices_needed = sample_size - min_true_count

    remaining_pool = np.setdiff1d(np.arange(array_A.size), selected_true_indices)

    selected_other_indices = np.random.choice(remaining_pool, remaining_indices_needed, replace=False)

    final_indices = np.concatenate((selected_true_indices, selected_other_indices))
    
    return final_indices

def calculate_row_entropy(df):
    def entropy(row):
        counts = row.value_counts(normalize=True)
        return -np.sum(counts * np.log2(counts + np.finfo(float).eps)) 

    return df.apply(entropy, axis=1)

def filter_array_by_range(array, min_value, max_value):
    # Create a boolean mask where True values are those within the specified range
    mask = (array >= min_value) & (array <= max_value)
    
    # Use the mask to select elements from the array
    return mask

def num_tokens_from_string(string, encoding_model = 'gpt-3.5-turbo') -> int:
    encoding = tiktoken.encoding_for_model(encoding_model)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [4]:
df2 = pd.read_csv('data/operation_patient_description.csv')
description_list = df2['description']

In [5]:
def generate_icl_sample(df, index, tasks = 'icu'):
    row = df.iloc[index]
    if tasks == 'icu':
        los_results = row.los_label
        readmin_label = row.readmin_label
        
        if los_results:
            # los_icl_text = 'The patient is discharged within 7 days of the admission. This indicates that the patient will be less likely to transferred to ICU (Predict as No)'
            los_icl_text = 'The patient is discharged within 7 days of the admission.'
        else:
            # los_icl_text = 'The patient remain in the hospital for more than 7 days. This indicates that the patient will be more likely to transferred to ICU (Predict as Yes)' 
            los_icl_text = 'The patient remain in the hospital for more than 7 days.'        
        if readmin_label:
            readmin_icl_text = 'The patient is readmitted to the hospital in 30 days.'
        else:
            readmin_icl_text = 'The patient is not readmitted to the hospital for the next 30 days.'

        icl_sample = f"**EHR Task Results Influencing whether the patient will be transferred to the ICU on the same admission date:**\nDischarge time: {los_icl_text}\nReadmission within 30 days: {readmin_icl_text}\n"
    elif tasks == 'los':
        icu_results = row.icu_label
        readmin_label = row.readmin_label

        if icu_results:
            icu_icl_text = 'The patient is transferred to the ICU within the same day of admission.'
        else:
            icu_icl_text = 'The paitent is not transferred to the ICU.'

        if readmin_label:
            readmin_icl_text = 'The patient is readmitted to the hospital in 30 days.'
        else:
            readmin_icl_text = 'The patient is not readmitted to the hospital for the next 30 days.'

        icl_sample = f"**ICL samples**\n**EHR Task Results Influencing whether the patient will be discharged within 7 days:**\nICU transfer: {icu_icl_text}\nReadmission within 30 days: {readmin_icl_text}\n"
    elif tasks == 'readmin':
        los_results = row.los_label
        icu_results = row.icu_label
        if los_results:
            los_icl_text = 'The patient is discharged within 7 days of the admission.'
        else:
            los_icl_text = 'The patient remain in the hospital for more than 7 days.'

        if icu_results:
            icu_icl_text = 'The patient is transferred to the ICU within the same day of admission.'
        else:
            icu_icl_text = 'The paitent is not transferred to the ICU on the admission day.'

        icl_sample = f"**EHR Task Results Influencing whether the patient will be readmitted in 30 days:**\nICU transfer: {icu_icl_text}\nDischarge time: {los_icl_text}\n"
    
    return icl_sample

In [6]:
# def combin_prompt_multitask(general_prompt, multitask_prompt, medical_event):
#     template_by_task = '**Task:**\n' + general_prompt + '\n'
    
#     template_events = '**Medical Events**\n' + medical_event + '\n'
    
#     multi_task_template = template_by_task + '\n' + multitask_prompt + '\n' + '{events}'
#     return multi_task_template, template_events

def combin_prompt_multitask(general_prompt, multitask_prompt, cot, medical_event):
    template_by_task = '**Task:**\n' + general_prompt + '\n'
    
    template_events = '**Medical Events**\n' + medical_event + '\n'

    template_cot = '**Chain of Thought Analysis:**\n'+ cot + '\n'

    multi_task_template = template_by_task + '\n' + multitask_prompt + '\n' + template_cot + '\n' + '{events}'
    return multi_task_template, template_events

In [7]:
# df2.iloc[10]
# a = generate_icl_sample(df2, 10, tasks='readmin')
# print(a)
df2 = pd.read_csv('data/operation_patient_description.csv')
description_list = df2['description']

In [8]:
prompt_list = ['prompts/cross_task/ICU_cross_task.txt', 'prompts/cross_task/LOS_cross_task.txt', 'prompts/cross_task/Readmin_cross_task.txt']
label_name = ['icu_label', 'los_label', 'readmin_label']
task = ['icu', 'los', 'readmin']
out_name = ['gpt35/icu_base']
cb_list = []
# for idx in range(1, 3):
for idx in [2]:
    out_csv_names = prompt_list[idx].split('/')[-1].replace('.txt', '.csv')
    general_prompt = generate_prompt_task_cross_task(prompt_list[idx])
    gt_array = df2[label_name[idx]]

    i_list = sample_indices(42, gt_array, 100, 20)

    gt_list = gt_array[i_list]

    task = label_name[idx].replace('_label', '')

    with get_openai_callback() as cb:

        answer_list_all = []

        for i in tqdm(i_list):
        # for i in tqdm(range(1)):
            answer_list_5 = []
            for _ in range(5):
                multitask_prompt = generate_icl_sample(df2, i, tasks=task)
                cot = generate_cot(tasks=task)
                c, e = combin_prompt_multitask(general_prompt, multitask_prompt, cot, description_list[i])

                prompt = ChatPromptTemplate.from_template(c)
                chain = prompt | llm
                result = chain.invoke({'events': e})
                answer_list_5.append(result.content)
            answer_list_all.append(answer_list_5)
            time.sleep(2)

        df_pred = pd.DataFrame(answer_list_all)
        df_pred.columns = ['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']
        df_pred['gt'] = gt_list.values
        df_pred['orig_index'] = i_list
        df_pred.to_csv(os.path.join('results/gpt4/cross_task/', prompt_list[idx].split('/')[-1].replace('.txt', '.csv')))
        cb_list.append(cb)

  0%|          | 0/100 [00:00<?, ?it/s]PID:97126 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net/openai/deployments/gpt-4-32k-beta/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:97126 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net/openai/deployments/gpt-4-32k-beta/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:97126 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net/openai/deployments/gpt-4-32k-beta/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:97126 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net/openai/deployments/gpt-4-32k-beta/chat/completions?api-version=2023-12-01-preview "HTTP/1

KeyboardInterrupt: 

In [9]:
# answer_list_all
answer_list_5

['Yes', 'Yes', 'Yes', 'Yes', 'Yes']

In [10]:
print(c)

**Task:**
You are an experienced doctor. Based on the provided patient age and medical events on the day of the admission, use your medical knowledge and reasoning to predict whether the patient will be readmitted in 30 days. Shortly explain your reasoning step by step, considering the critical factors that would influence such a decision. 
Please only answer with Yes or No and do not output the reasoning parts. Yes if the patient will be readmitted in 30 days, No, if the patient will not be readmitted in 30 days.
Also, you are given the patient's results of EHR tasks on "ICU transfer" (whether the patient will be transferred to ICU on admission date) and "Discharge time" (whether the patient will be discharged in 7 days), provided as EHR tasks references.

**EHR Task Results Influencing whether the patient will be readmitted in 30 days:**
ICU transfer: The paitent is not transferred to the ICU on the admission day.
Discharge time: The patient remain in the hospital for more than 7 day

In [18]:
cb_list[0]

Tokens Used: 2203390
	Prompt Tokens: 2202390
	Completion Tokens: 1000
Successful Requests: 500
Total Cost (USD): $132.26339999999956

In [11]:
# df2.loc[i_list, ['icu_label', 'los_label', 'readmin_label']].to_csv('sample.csv')

In [12]:
df2

,patient_id,admission_date,discharge_date,delta_days,description,icu_label,los_label,readmin_label
0,115973157,2022-12-09 23:59:00,2022-12-12 23:59:00,3,"The patient was 75 years old.\nIn December 09,...",False,False,False
1,115973640,2012-11-13 23:59:00,2012-11-15 23:59:00,2,"The patient was 77 years old.\nIn December 09,...",False,False,False
2,115973136,2014-10-01 23:59:00,2014-10-03 23:59:00,2,"The patient was 32 years old.\nIn December 09,...",False,False,False
3,115973151,2019-08-05 23:59:00,2019-08-09 23:59:00,4,"The patient was 36 years old.\nIn December 09,...",False,False,False
4,115973694,2020-08-11 23:59:00,2020-08-16 23:59:00,5,"The patient was 62 years old.\nIn December 09,...",False,False,False
...,...,...,...,...,...,...,...,...
2374,115967163,2009-11-10 23:59:00,2009-11-13 23:59:00,3,"The patient was 65 years old.\nIn December 09,...",False,False,False
2375,115967174,2016-04-15 23:59:00,2016-04-17 23:59:00,2,"The patient was 37 years old.\nIn December 09,...",False,False,False
2376,115967235,2017-09-26 23:59:00,2017-09-30 23:59:00,4,"The patient was 38 years old.\nIn December 09,...",False,False,False
2377,115967238,2016-04-29 23:59:00,2016-05-01 23:59:00,2,"The patient was 43 years old.\nIn December 09,...",False,False,False


In [13]:
df_pred

,pred_1,pred_2,pred_3,pred_4,pred_5,gt,orig_index
0,No,No,No,No,No,True,257
1,Yes,Yes,Yes,Yes,No,True,538
2,Yes,Yes,Yes,Yes,Yes,True,1792
3,Yes,Yes,Yes,Yes,Yes,True,442
4,Yes,Yes,Yes,Yes,Yes,True,803
...,...,...,...,...,...,...,...
95,Yes,Yes,No,Yes,Yes,False,1081
96,Yes,Yes,Yes,Yes,Yes,False,242
97,Yes,Yes,Yes,Yes,Yes,False,2023
98,No,Yes,Yes,Yes,No,False,135


In [14]:
e

'**Medical Events**\nThe patient was 31 years old.\nIn December 09, 2022, at the time of the admission:\n1 condition_occurrence medical events (Nausea and vomiting) occurred. \n1 visit_occurrence medical events (Plan Stop Reason - Reason for termination of the Health Plan) occurred. \n1 condition_occurrence medical events (Constipation) occurred. \n1 condition_occurrence medical events (Immunodeficiency disorder) occurred. \n1 condition_occurrence medical events (Benign neoplastic disease) occurred. \n1 observation medical events (Antineoplastic adverse reaction) occurred. \n2 observation medical events (Tobacco user) occurred. \n2 observation medical events (Snuff consumption) occurred. The values recorded were: N. \n2 observation medical events (Chewed tobacco consumption) occurred. The values recorded were: N. \n2 observation medical events (Cigarette consumption) occurred. The values recorded were: N. \n2 observation medical events (Pipe tobacco consumption) occurred. The values re